In [48]:
%load_ext autoreload
%autoreload 2

import os
from oads_access.oads_access import OADS_Access, OADSImageDataset
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from pytorch_utils.pytorch_utils import collate_fn
import tqdm
from torchvision import transforms
import time, torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
home_dir = os.path.expanduser('~') 
oads = OADS_Access(os.path.join(home_dir, 'projects', 'data', 'oads'))
data_dir = os.path.join(home_dir, 'projects', 'data', 'oads', 'oads_arw', 'crops', 'ML')

In [32]:
oads.image_names.keys()

dict_keys(['61b1e87959b6b6f0', 'bbbdccca89b876e4', 'cccc8c2c2dcb4238', 'dcecb4f0f1a38306', 'e296981a1aca183c', 'b099d9787019b0e2', '8ccdcccc46e4c0f0', '586c2c1c9415c2ec', 'e9c0c0f0b0b03971', 'b490d0e2e68c12f8', 'b2b2b3b3b292c1e1', 'c0d4dcccfc830e1c', 'b7664acfce263f3e', '90686d2424a0c0d0', '5d7f69185dfdf89c', 'd1c3d3d188b4e688', '9a9a9adadcfdfcb8', 'c1c5c5cdde94c484', '5858babc3938f6c6', 'b1f1b08c28741c81', 'd9d0d1e8f0d3d2e6', '968787539aad8bcf', '557456d62d2d07b6', 'e2c6c4d0d2d2f2c0', '49036343d2609a98', '32f226627134b1d8', '99a4b6ac6d6d6d9f', 'b0b0b7af97913070', 'b031319a8ad3a3e0', 'e0e0e6e3a38deff2', 'c1c8d9d90c90dcdc', 'a5b9ddc6c6c3edfc', '8090e266b6e6cfce', '7134a6e9f278d85a', '647666666692c0e2', '70636666787246cc', '987d646a4849c4c0', 'c983c1c0a0ac64e4', '5a3bf1c069e1e5c1', '90921b2b2262e4cc', '840424212365d890', '96961719d9fa9e89', 'b4b8f05138a6b6b8', '6062646534a464c4', 'aeb2919fbcf54fc8', '98908404180230b0', 'd2c2c1c0da62e1f3', '185838b444e68206', 'ce4f8b879ab2dd59', 'e120a10d

In [35]:
_,_,ids = oads.get_train_val_test_split_indices(use_crops=True)

In [36]:
print(ids)

[('bca7b0bc60e3f7b0', 9), ('f8cbcf4cc9842434', 9), ('b1b49da5a27360e4', 11), ('2033237557fb7ef8', 7), ('63cef2ea3a70f0b8', 2), ('e1f1e44585f0c8e0', 10), ('87e6e626d80286c0', 2), ('80909e0c4d8fb833', 7), ('7c733b3332268df8', 4), ('49c47461796be4c8', 17), ('b9b130b03090d8f1', 24), ('189a5262d2c0c890', 25), ('a4a0328a91b0a30b', 32), ('3839b28d85a4a0b0', 12), ('9c9e5d9a9a397878', 1), ('b9b51094868080e0', 10), ('e4e6cbc64e62cb8b', 9), ('8cc8f9d0d33766f0', 9), ('d4dcd4c6c9c8f5fa', 8), ('a26623a726f2c0c0', 14), ('7edeb7e9a7a5e2c6', 8), ('cffbcfcbcad69efc', 3), ('88e06060603060e0', 7), ('928a1026a0e88684', 9), ('92004cb6466060b0', 5), ('dbb3b7a530f0e0b0', 4), ('7341b4c69ff8e8e2', 2), ('e18046450591e3c2', 15), ('8fcfc9c8c6d6483e', 5), ('ce46460ad6a5b0d8', 0), ('b7b7b5971c70e0c0', 1), ('bb3e793b736efef8', 10), ('a3a08166b080e0f0', 16), ('d0e2e5d7d8fcd2d0', 8), ('fafefaf25ede3770', 3), ('83cba4289a3320f8', 1), ('2c3424b1b07848f0', 32), ('787878d9c9d931ce', 8), ('c1c6e4d1929ae4f4', 22), ('c5593045

In [39]:
start = time.time()
for name, index in ids[:10]:
    img, lbl = oads.load_crop_from_image(image_name=name, index=index)

end = time.time()

print(end-start)

start = time.time()
for name, index in ids[:10]:
    img, lbl = oads.load_crop_from_image(image_name=name, index=index, force_recompute=True)

end = time.time()

print(end-start)

6.719647169113159
44.406301975250244


In [41]:
imsize = 400

mean = [0.3410, 0.3123, 0.2787]
std = [0.2362, 0.2252, 0.2162]

transform = transforms.Compose([
    transforms.Resize((imsize, imsize)),  # scale imported image
    transforms.ToTensor(),
    transforms.Normalize(mean, std)])  # transform it into a torch tensor

In [40]:
output_channels = len(oads.get_class_mapping())
class_index_mapping = {}
index_label_mapping = {}
for index, (key, item) in enumerate(list(oads.get_class_mapping().items())):
    class_index_mapping[key] = index
    index_label_mapping[index] = item

In [50]:
device = torch.device('cuda:1')
batch_size = 4

In [53]:
# if bool(args.preload_all):
#     traindataset = OADSImageDatasetSharedMem(oads_access=oads, item_ids=train_ids, use_crops=True, size=(n_input_channels, size[0], size[1]),
#                                     class_index_mapping=class_index_mapping, transform=transform, device=device)
#     valdataset = OADSImageDatasetSharedMem(oads_access=oads, item_ids=val_ids, use_crops=True, size=(n_input_channels, size[0], size[1]),
#                                     class_index_mapping=class_index_mapping, transform=transform, device=device)
#     testdataset = OADSImageDatasetSharedMem(oads_access=oads, item_ids=test_ids, use_crops=True, size=(n_input_channels, size[0], size[1]),
#                                     class_index_mapping=class_index_mapping, transform=transform, device=device)
# else:
# traindataset = OADSImageDataset(oads_access=oads, item_ids=train_ids, use_crops=True,
#                                 class_index_mapping=class_index_mapping, transform=transform, device=device)
# valdataset = OADSImageDataset(oads_access=oads, item_ids=val_ids, use_crops=True,
#                                 class_index_mapping=class_index_mapping, transform=transform, device=device)
testdataset = OADSImageDataset(oads_access=oads, item_ids=ids, use_crops=True,
                                class_index_mapping=class_index_mapping, transform=transform, device=device)

# Create loaders - shuffle training set, but not validation or test set
# trainloader = DataLoader(traindataset, collate_fn=collate_fn,
#                             batch_size=batch_size, shuffle=True, num_workers=oads.n_processes)
# valloader = DataLoader(valdataset, collate_fn=collate_fn,
#                         batch_size=batch_size, shuffle=False, num_workers=oads.n_processes)
testloader = DataLoader(testdataset, collate_fn=collate_fn,
                        batch_size=batch_size, shuffle=False, num_workers=4)

In [11]:
# dataset = ImageFolder(root=data_dir, transform=oads_transform)
# loader = DataLoader(dataset, batch_size=8, shuffle=True)

In [46]:
len(testloader.dataset)

8643

In [54]:
start = time.time()

for item in tqdm.tqdm(testloader):
    x = item[0]
    y = item[1]

    x = x.to(device)
    y = y.to(device)

end = time.time()

print(end - start)

  0%|          | 0/2161 [00:36<?, ?it/s]


KeyboardInterrupt: 